In [ ]:
import numpy as np
from astropy.table import Table
import pandas as pd
from matplotlib import pyplot as plt
from scipy.signal import correlate
from scipy.stats import pearsonr, poisson, chi2

import seaborn as sns
from matplotlib import colors

import warnings
warnings.filterwarnings('ignore')

# Load data

In [ ]:
# Load data from raw csv. The csv files used here and the xlsx files used in the R scripts are identical.
# The data has two patches, one for updates sepsis onset data and one for updated microbiology
data = pd.read_csv("./data/data.csv")
datas = pd.read_csv("./data/data_sepsis_update.csv")
datam = pd.read_excel('./data/data_micro_update.xlsx', engine="openpyxl")

# Two aberration alarms are computed using Aberrations.Rmd and should be loaded here
alarmss = np.loadtxt('./data/alarmss.txt')
alarmsd = np.loadtxt('./data/alarmsd.txt')

# Fill every na value with 0s
data = data.fillna(0)
datas = datas.fillna(0)
datam = datam.fillna(0)

# Load Tables
data = Table.from_pandas(data)
datas = Table.from_pandas(datas)
datam = Table.from_pandas(datam)

# Select only relevant weeks. 
data = data[182:365]
datas = datas[182:365]
datam = datam[182:365]

# Make sure the relevant weeks are actually correct. 
print(data['epiwk'][0], datas['epiwk'][0], datam['epiwk'][0])
print(data['epiwk'][-1], datas['epiwk'][-1], datam['epiwk'][-1])
print(len(data), len(alarmss))

# Helper functions

In [ ]:
def ma(x, w=5, sim=True, future=False):
    #Moving average in a window of size w
    #if sim is true then the moving average at index 0 uses the values at indexes [-w/2, ..., 0, ...,w/2]
    #if future is true, then the ma at index 0 uses the values at indexes [0, ..., w]
    
    if(sim):
        return np.convolve(x, np.ones(w), 'same') / w
    if(future):
        return np.convolve(x, np.concatenate([np.ones(w),[0],np.zeros(w)]), 'same') / w
    return np.convolve(x, np.concatenate([np.zeros(w),[0],np.ones(w)]), 'same') / w

def pr(x, y):
    # Computes the correlation coefficient for two arrays x and y
    
    idx = np.isfinite(x) & np.isfinite(y)
    x, y = x[idx], y[idx]
    return pearsonr(x, y)


def get_pvalue(ats, its, n=20000, v=True, laba = '', labi='indicator'):
    # Quantifies the effectiveness of an alarm time series.
    #
    # ats the alarm time series 
    # its the indicator time series
    # n how many samples to take, the minimum p-value is around 1/n
    # v verbose, print the time series. 
    #
    # returns the p-value or the alarm-indicator combination provided. 
    
    idxa = (ats != 0)
    mean = np.mean(its[idxa])
    num = idxa.sum()
    
    m = np.zeros(n)
    for i in range(n):
        m[i] = np.mean(np.random.choice(its, num, replace=False))
    
    if(v):
        
        # Plots the indicator and alarm on top of each other
        plt.title(laba+' alarm ('+str(num)+' weeks)')
        plt.plot(its, c='k')
        plt.vlines(np.arange(len(ats))[ats!=0], its.min(), its.max(), colors='r')
        plt.axhline(mean, ls='--', c='k')
        plt.gca().set_ylabel(labi)
        plt.gca().set_xlabel('week number')
        plt.show()
        
        # Plots the indicator and a random alarm on top of it
    
        random_alarm_indices = np.random.permutation(np.arange(len(ats)))[:num]
        
        random_alarm = ats[random_alarm_indices]
        mean_random = np.mean(its[random_alarm_indices])
        plt.title('random alarm ('+str(num)+' weeks)')
        plt.plot(its, c='k')
        plt.vlines(random_alarm_indices, its.min(), its.max(), colors='b')
        plt.axhline(mean_random, ls='--', c='k')
        plt.gca().set_ylabel(labi)
        plt.gca().set_xlabel('week number')
        plt.show()

        # plot the distribution of the values

        plt.title('distribution of average '+labi+' in '+str(num)+' weeks')
        plt.hist(m, fc='0.5')
        plt.axvline(mean, label=laba+' alarm', c='r')
        plt.axvline(mean_random, label='random alarm', c='b')
        plt.legend()
        plt.show()
    
    return ((m > mean).sum()/n)
    
    
def get_randoms(itsarray, w=20, n=10000):
    # Simulates a random alarm and returns the average of multiple indicators
    # in the weeks selected by the said alarm. The operation can be iterated n times
    # its indicators array
    # w number of weeks the simulated alarm was called
    # n how many times to produce a random alarm
    #
    # returns an array with 2 dimensions, one index runs through the indicators, 
    # the other one through the multiple realizations
    
    m = np.zeros((len(itsarray), n))
    itsarray = np.array(itsarray)
    
    for i in range(n):
        idxs = np.random.choice(np.arange(itsarray.shape[1]), w, replace=False)
    
        m[:, i] = np.mean(itsarray[:, idxs], axis=1)
        
    return m
    

def get_corr(dat, v=False):
    # Calculates the correlation matrix for a set of indicators. 
    #
    # returns an array containing the correlation matrix and the p-value matrix. 
    
    n = dat.shape[0]
    res = np.zeros((n, n))
    res2 = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            res[i, j] = pearsonr(dat[i, :], dat[j, :])[0]
            res2[i, j] = pearsonr(dat[i, :], dat[j, :])[1]
    
    return res, res2




def get_sensitivity(alarms):
    # Given a series of alarms, it computes the sensitivity matrix, res.
    # res[i, j] is the sensitivity of alarm j to alarm i
    
    n = len(alarms)
    alarms = np.array(alarms)
    
    res = np.zeros((n, n))
    for i,a1 in enumerate(alarms):
        for j, a2 in enumerate(alarms):
            res[i, j] = (a1*a2 != 0).sum()/(a1 != 0).sum()
    return res


def get_specificity(alarms):
    # Given a series of alarms, it computes the specificity matrix. See above.
    
    n = len(alarms)
    alarms = np.array(alarms)
    
    res = np.zeros((n, n))
    for i,a1 in enumerate(alarms):
        for j, a2 in enumerate(alarms):
            res[i, j] = ((a1==0)*(a2==0)).sum()/(a1 == 0).sum()
    return res


def get_ppv(alarms):
    # Given a series of alarms, it computes the positive predictive value matrix. See above.
    
    n = len(alarms)
    alarms = np.array(alarms)
    
    res = np.zeros((n, n))
    #a1 is the gold standard
    for i,a1 in enumerate(alarms):
        for j, a2 in enumerate(alarms):
            res[i, j] = (a1*a2 != 0).sum()/((a2 != 0)).sum()
    return res    

def get_npv(alarms):
    # Given a series of alarms, it computes the negative predictive value matrix. See above.
    
    n = len(alarms)
    alarms = np.array(alarms)
    
    res = np.zeros((n, n))
    #a1 is the gold standard
    for i,a1 in enumerate(alarms):
        for j, a2 in enumerate(alarms):
            res[i, j] = ((a1==0)*(a2==0)).sum()/((a2 == 0)).sum()
    return res    


def get_mcN_test(alarms):
    # Given a series of alarms, quantifies the compatibility of their specificity by performing a McNemar's test. 
    # The output are matrices definited similarly to the quantities defined above. 
    #
    # returns the p-value and the numbers of false positives. 
        
    n = len(alarms)
    alarms = np.array(alarms)
    
    res = np.zeros((n, n))
    res2 = np.zeros((n, n))
    res3 = np.zeros((n, n))
    #a1 is the gold standard
    for i,a1 in enumerate(alarms):
        for j, a2 in enumerate(alarms):
            part1 = ((a1==0) & (a2!=0)).sum() # pos-neg
            part2 = ((a1!=0) & (a2==0)).sum() # neg-pos
            chi2_rep = (part1-part2)**2./(part1+part2)
            res[i, j] = 1.-chi2.cdf(chi2_rep, df=1) 
            res2[i, j] = part2
            res3[i, j] = part1
    return res, res2, res3  


# Plotting function
def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
    rng = M - m
    norm = colors.Normalize(m - (rng * low),
                            M + (rng * high))
    normed = norm(s.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]

# Define alarm time series


See article to know how alarm_LOS_threshold, alarm_LOS_rate, alarm_mortality, alarm_LOS_diff, alarm_death_diff are defined.

In [ ]:
ats = np.zeros(len(data))
ats[datas['sepsis_los_adm']>4] = 1
alarm_LOS_threshold = ats

ats = np.zeros(len(data))
ats[datas['sepsis_los_adm']/data['neo_adm']>0.1] = 1
alarm_LOS_rate = ats

ats = np.zeros(len(data))
ats[data['neo_death']/data['neo_exit']>0.2]= 1
alarm_mortality = ats 

pctincrease = np.zeros(len(data))
pctincrease[1:] = (datas['sepsis_los_adm'][1:]-datas['sepsis_los_adm'][:-1])/datas['sepsis_los_adm'][:-1]
pctincrease[~np.isfinite(pctincrease)] = 0
ats = np.zeros(len(data))
ats[ pctincrease > 1. ] = 1
alarm_LOS_diff = ats 


pctincrease = np.zeros(len(data))
pctincrease[1:] = (data['neo_death'][1:]-data['neo_death'][:-1])/data['neo_death'][:-1]
pctincrease[~np.isfinite(pctincrease)] = 0
ats = np.zeros(len(data))
ats[ pctincrease > 1. ] = 1
alarm_death_diff = ats 


# This is the alarms list
alarms = [alarm_LOS_threshold, 
          alarm_LOS_rate,
          alarmss,
          alarm_mortality,
          alarmsd,
          alarm_LOS_diff,
          alarm_death_diff]


# These are the alarm labels
als = ['LOS threshold', 
       'LOS rate \n  threshold',
       'LOS rate \n aberration',
       'Mortality \n threshold',
       'Mortality \n aberration',
       'LOS cases \n differential', 
       'Deaths \n differential']

#als = [al+"\n n="+str((alarms[i] != 0).sum()) for i, al in enumerate(als)]

# Save the alarms for Plotting.Rmd
np.savetxt("data/alarmscomplete_from_py.txt", np.array(alarms, dtype=int).T, fmt='%i', delimiter=",")

# Evaluates consistency between some alarms by computing five different metrics.

Sensitivity, specificity, positive predictive value, negative predictive value, McNemar's test. Note that we also simulate an alarm related to the microbiology. The result of this snippet are multiple matrices describing the results of these tests.

In [ ]:

# Grab only the alarms used in this section
alarms2 = [alarm_LOS_threshold, 
          alarm_LOS_rate,
          alarmss,
          alarm_mortality,
          alarmsd,
          datam['gn_bc_sample']]

# Define some labels
als2 = ['LOS threshold', 
       'LOS rate \n threshold',
       'LOS rate \n aberration',
       'Mortality \n threshold',
       'Mortality \n aberration',
       'Gram-negative bacteria \n positive blood cultures']


# Compute the matrices using the helpes above
overlap = get_sensitivity(alarms2)
spec = get_specificity(alarms2)
ppv = get_ppv(alarms2)
npv = get_npv(alarms2)
mcN_test, mcN_check, mcN_check2 = get_mcN_test(alarms2)

# Define some labels for the matrices
labels_matrices = ['Sensitivity',
                  'Specificity',
                  'PPV',
                  'NPV',
                  'McNemar\'s test p-value']

matrices = [overlap, spec, ppv, npv, mcN_test]

# Create nice pandas matrices
overlap = pd.DataFrame(data=np.round(overlap*100), columns=als2, index=als2)
spec = pd.DataFrame(data=np.round(spec*100), columns=als2, index=als2)
ppv = pd.DataFrame(data=np.round(ppv*100), columns=als2, index=als2)
npv = pd.DataFrame(data=np.round(npv*100), columns=als2, index=als2)
mcN_test = pd.DataFrame(data=mcN_test, columns=als2, index=als2)
mcN_check = pd.DataFrame(data=mcN_check, columns=als2, index=als2)
mcN_check2 = pd.DataFrame(data=mcN_check2, columns=als2, index=als2)

for label_matrix, matrix in zip(labels_matrices, matrices):
    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(5.5, 4.5))

    # Generate a custom diverging colormap
    cmap = sns.color_palette("Blues", as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(matrix, square=False, cmap=cmap, linewidths=.5,  annot=True)
    plt.title(label_matrix)
    pos, textvals = plt.yticks()
    plt.yticks(pos,textvals, 
        rotation=0, fontsize="10", va="center")
    plt.show()




# Define indicator time series

See article for the definitions of indicator_los, indicator_los_future, indicator_mortality, indicator_mortality_future, indicator_bc (blood cultures), indicator_antibiotics, indicator_bor (bed occupancy rate), indicator_maternal_exit,  indicator_maternal_normal (maternal admissions), indicator_neo (neonatal admissions).

In [ ]:
indicator_los = ma(datas['sepsis_los_adm'])/ma(data['neo_adm'])

indicator_los_future = ma(datas['sepsis_los_adm'], future=True, sim=False, w=3)/ma(data['neo_adm'], future=True, sim=False, w=3)
indicator_los_future[-1] = 0

indicator_mortality = ma(data['neo_death'])/ma(data['neo_exit'])

indicator_mortality_future = ma(data['neo_death'], future=True,sim=False, w=3)/ma(data['neo_exit'], future=True, sim=False, w=3)
indicator_mortality_future[-1] = 0

indicator_bc = ma(datam['gn_bc_sample'])

indicator_antibiotics = data['mero_total']+data['cefta_total'] + data['amik_total']

indicator_bor = data['bor']

indicator_maternal_exit = ma(data['mat_exit'])

indicator_maternal_normal = ma(data['mat_normal_preg'])

indicator_neo = ma(data['neo_adm'])


# Group the indicators in an array
indicators = [indicator_los, 
              indicator_mortality, 
              indicator_mortality_future, 
              indicator_los_future,
              indicator_bc, 
              indicator_antibiotics, 
              indicator_maternal_exit,
              indicator_maternal_normal,
              indicator_neo]

# And give them labels
ils = ['LOS cases', 
       'Mortality', 
       'Future \n mortality', 
       'Future \n LOS cases',
       'GNB positive \n blood cultures', 
       '2nd & 3rd line \n antibiotic \n consumption', 
       'Maternal exits',
       'Normal \n pregnancy\n admissions',
      'Neonatal\n admissions']

indicators_to_save = [indicator_los, 
              indicator_mortality, 
              indicator_mortality_future, 
              indicator_los_future,
              indicator_bc]

np.savetxt("data/indicatorscomplete_from_py.txt", np.array(indicators_to_save).T, delimiter=",")



# Compute the correlation between different indicators

In [ ]:

# To compute the correlation of different indicators, it is ideal to average over multiple weeks.
# This is because the resulting distributions are Normal. For heavily skewed distributions the meaning of the
# correlation coefficient is not clear. 
#This assumption is relaxed in the final snippet of this notebook. 
dat = get_randoms(indicators, w=25)

# Compute the correlation matrix using this random selection
corr, pvals = get_corr(dat)

# make it into a pandas dataframe
corr = pd.DataFrame(data=corr,
                 columns=ils, index=ils)

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(8, 7))

# Generate a custom diverging colormap
cmap = sns.color_palette("Blues", as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, cmap=cmap, linewidths=1, annot=True, fmt='.1g', cbar=False)

#Plotting style
plt.title('Correlation matrix for 20-week average value of the indicators')

pos, textvals = plt.yticks()
plt.yticks(pos,textvals, 
    rotation=0, fontsize="12", va="center")

pos, textvals = plt.xticks()
plt.xticks(pos,textvals, 
    rotation=90, fontsize="12")
plt.gca().xaxis.tick_top()

plt.show()

# Compute the p-value quantifying if the alarm can predict the indicator and plot them

This takes a few minutes since 2e4 realizations are simulated

In [ ]:
p = np.zeros((len(als), len(ils)))

for i, at in enumerate(alarms):
    for j, it in enumerate(indicators):
        p[i, j] = get_pvalue(at, it, v=False)
        
p = pd.DataFrame(data=p,
                 columns=ils, index=als)

In [ ]:

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))


sns.heatmap(p,
            mask=(p !=0),
            linewidth=0.5,
            annot=np.array([r'$\mathbf{<0.001}$'
                            for data in np.array(p).ravel()]).reshape(
                np.shape(p)),
            fmt='',
            cbar=False)

sns.heatmap(p,
            mask=(p >0.021) | (p==0),
            linewidth=0.5,
            annot=np.array([r'$\mathbf{'+str(data)+'}$'
                            for data in np.array(p).ravel()]).reshape(
                np.shape(p)),
            fmt='',
            cbar=False)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(p, mask=p<0.021, square=False, linewidths=.5, annot=True, fmt='.1g',cbar=False, cmap=['0.9'])




pos, textvals = plt.yticks()
plt.yticks(pos,textvals, 
    rotation=0, fontsize="10", va="center")


pos, textvals = plt.xticks()
plt.xticks(pos,textvals, 
    rotation=45, fontsize="10")
plt.gca().xaxis.tick_top()


plt.tight_layout()
plt.show()

# Example of how this p-value is generated

The plots below are an example of how the p-values above are obtained for a combination of indicator and alarm. There are three figures.

1) The selected indicator time series with the weeks marked by the selected alarm overplotted. The y-label indicates the indicator name, the plot title mentions the alarm name. The dashed horizontal line marks the average value of the indicator in the weeks selected. This average acts as a summary statistics for the indicator-alarm combination. 

2) The same as the plot above only with a "random alarm", i.e. a random selection of the same number of weeks as the real alarm. 

3) These two averages are plotted together with a histogram of the possible values of the blue line over multiple realizations of plot 2.  Because it is expected to be a Normal distribution, there is no need to obtain the value of the summary statistics over every possible combination of random alarm weeks. 

In [ ]:
res = get_pvalue(alarms[0], indicators[1], n=20000, v=True, laba = als[0], labi=ils[1]+" (%)")

The final image also shows how the average of an indicator over many weeks follows a normal distribution, meaning that the correlation factors between two indicator averages has a meaningful interpretation. 

Below we plot the correlation matrix between average values and between the simple time-series values of the indicators. The two have the same correlation factors, but the significance of the correlation is different, because the test for Pearson's r assumes a Normal distribution. 

In [ ]:
# This is simply a collection of the indicators.
# data_not_avg[i, j], i runs through the indicators, j though the week number
# Only two indicators are used here for simplicity.
data_not_avg = np.array(indicators)

# get_randoms is used to get the averages over 25 weeks. 
# data_avg[i, j], i runs through the indicators, j through the realizations of the averages. 
# For example, data_avg[1, j] is what forms the histogram above. 
data_avg = get_randoms(indicators, w=25)


# Compute the correlation matrix using the random selection
corr_avg, p_avg = get_corr(data_avg)

# Compute the correlation matrix using the time-series selection
corr_not_avg, p_not_avg = get_corr(data_not_avg)

In [ ]:
titles = ["Correlation coefficients (CCs) for average indicators",
         "Correlation coefficients (CCs) for time series",
         "p-value of the CCs for the averages",
         'p-value of the CCs for the time series']


for title, matrix in zip(titles, [corr_avg, corr_not_avg, p_avg, p_not_avg]):
    matrix = pd.DataFrame(data=matrix,
                 columns=ils, index=ils)
    plt.subplots(figsize=(8, 6))
    plt.title(title)
    sns.heatmap(matrix, annot=True)
    pos, textvals = plt.yticks()
    plt.yticks(pos,textvals, 
        rotation=0, fontsize="12", va="center")

    pos, textvals = plt.xticks()
    plt.xticks(pos,textvals, 
        rotation=90, fontsize="12")
    plt.gca().xaxis.tick_top()

    plt.show()